In [ ]:
#tss
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import numpy as np

# Function to save MAPE for each country
def save_country_mape(country_mape, target_name):
    country_mape.to_csv(f'{target_name}_country_mape.csv', index=True, header=['MAPE'])

# Load the data
data = pd.read_csv('TSS_C.csv')

# Extract the 'Country' column
countries = data['Country']

# Select columns 2 to 10 for imputation
columns_for_imputation = data.columns[2:11]

# Impute NaN values for columns 2 to 10 using KNN
imputer = KNNImputer(n_neighbors=5)
data_imputed = data.copy()
data_imputed[columns_for_imputation] = imputer.fit_transform(data[columns_for_imputation])

# Check for and handle remaining NaN values
if data_imputed[columns_for_imputation].isnull().values.any():
    raise ValueError("NaN values still present after imputation.")

# Separate features and target variables
X = data_imputed.drop(columns=['Country', 'TSS'])
y_tss = data_imputed['TSS']

# Standardize the features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Define the MDN loss function
num_components = 1
def mdn_loss(y_true, y_pred):
    # Extract mean, log_sigma, and alpha using indexing
    num_params = num_components * 3
    mean = y_pred[:, :num_components]
    log_sigma = y_pred[:, num_components:2*num_components]
    alpha = y_pred[:, 2*num_components:]

    sigma = K.exp(log_sigma)

    # Gaussian probability density function
    pdf = K.exp(-0.5 * K.square((y_true - mean) / (sigma + 1e-8))) / (sigma + 1e-8) / np.sqrt(2 * np.pi)

    # Mixture of Gaussians
    loss = -K.log(K.sum(alpha * pdf, axis=1, keepdims=True) + 1e-8)
    return loss

# Function to train MDN model for each country and save MAPE
def train_and_save_mape(X, y, target_name):
    country_mape = pd.DataFrame(index=countries.unique())

    for country in countries.unique():
        # Filter data for the current country
        country_mask = (countries == country)
        X_country = X[country_mask]
        y_country = y[country_mask]

        # Create and compile the MDN model
        mdn_model = create_mdn_model()

        # Implement early stopping
        early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

        # Train the MDN model on the current country's data
        mdn_model.fit(
            X_country, y_country,
            epochs=100,  # You can adjust the number of epochs
            batch_size=32,
            callbacks=[early_stopping],
            verbose=1
        )

        # Generate predictions on the MDN model
        y_pred = mdn_model.predict(X_country)

        # Calculate and save MAPE for the current country
        mape = mean_absolute_percentage_error(y_country, y_pred[:, :1].flatten())  # Use mean for prediction
        country_mape.loc[country, 'MAPE'] = mape

    # Save MAPE for each country as a CSV
    save_country_mape(country_mape, target_name)

# Train and save MAPE for TSS
train_and_save_mape(X_scaled, y_tss, 'TSS')




Epoch 1/100
5/5 [==============================] - 1s 6ms/step - loss: nan
Epoch 2/100
5/5 [==============================] - 0s 4ms/step - loss: nan
Epoch 3/100
5/5 [==============================] - 0s 4ms/step - loss: nan
Epoch 4/100
5/5 [==============================] - 0s 4ms/step - loss: nan
Epoch 5/100
5/5 [==============================] - 0s 4ms/step - loss: nan
Epoch 6/100
5/5 [==============================] - 0s 5ms/step - loss: nan
Epoch 7/100
5/5 [==============================] - 0s 4ms/step - loss: nan
Epoch 8/100
5/5 [==============================] - 0s 4ms/step - loss: nan
Epoch 9/100
5/5 [==============================] - 0s 4ms/step - loss: nan
Epoch 10/100
5/5 [==============================] - 0s 3ms/step
Epoch 1/100
1/1 [==============================] - 1s 970ms/step - loss: 6.0326
Epoch 2/100
1/1 [==============================] - 0s 17ms/step - loss: 5.8204
Epoch 3/100
1/1 [==============================] - 0s 14ms/step - loss: 5.6504
Epoch 4/100
1/1 [======

1/1 [==============================] - 0s 55ms/step
Epoch 1/100
4/4 [==============================] - 1s 4ms/step - loss: nan
Epoch 2/100
4/4 [==============================] - 0s 5ms/step - loss: nan
Epoch 3/100
4/4 [==============================] - 0s 6ms/step - loss: nan
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: nan
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: nan
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: nan
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: nan
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: nan
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: nan
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: nan


4/4 [==============================] - 0s 3ms/step
Epoch 1/100
6/6 [==============================] - 1s 5ms/step - loss: nan
Epoch 2/100
6/6 [==============================] - 0s 3ms/step - loss: nan
Epoch 3/100
6/6 [==============================] - 0s 4ms/step - loss: nan
Epoch 4/100
6/6 [==============================] - 0s 3ms/step - loss: nan
Epoch 5/100
6/6 [==============================] - 0s 3ms/step - loss: nan
Epoch 6/100
6/6 [==============================] - 0s 3ms/step - loss: nan
Epoch 7/100
6/6 [==============================] - 0s 3ms/step - loss: nan
Epoch 8/100
6/6 [==============================] - 0s 3ms/step - loss: nan
Epoch 9/100
6/6 [==============================] - 0s 3ms/step - loss: nan
Epoch 10/100
6/6 [==============================] - 0s 3ms/step
Epoch 1/100
4/4 [==============================] - 1s 4ms/step - loss: nan
Epoch 2/100
4/4 [==============================] - 0s 3ms/step - loss: nan
Epoch 3/100
4/4 [==============================] - 0s 4ms/st

In [ ]:
#chla
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import numpy as np

# Function to save MAPE for each country
def save_country_mape(country_mape, target_name):
    country_mape.to_csv(f'{target_name}_country_mape.csv', index=True, header=['MAPE'])

# Load the data
data = pd.read_csv('Chla_C.csv')

# Extract the 'Country' column
countries = data['Country']

# Select columns 2 to 10 for imputation
columns_for_imputation = data.columns[2:11]

# Impute NaN values for columns 2 to 10 using KNN
imputer = KNNImputer(n_neighbors=5)
data_imputed = data.copy()
data_imputed[columns_for_imputation] = imputer.fit_transform(data[columns_for_imputation])

# Check for and handle remaining NaN values
if data_imputed[columns_for_imputation].isnull().values.any():
    raise ValueError("NaN values still present after imputation.")

# Separate features and target variables
X = data_imputed.drop(columns=['Country', 'Chla'])
y_tss = data_imputed['Chla']

# Standardize the features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Define the MDN loss function
num_components = 1
def mdn_loss(y_true, y_pred):
    # Extract mean, log_sigma, and alpha using indexing
    num_params = num_components * 3
    mean = y_pred[:, :num_components]
    log_sigma = y_pred[:, num_components:2*num_components]
    alpha = y_pred[:, 2*num_components:]

    sigma = K.exp(log_sigma)

    # Gaussian probability density function
    pdf = K.exp(-0.5 * K.square((y_true - mean) / (sigma + 1e-8))) / (sigma + 1e-8) / np.sqrt(2 * np.pi)

    # Mixture of Gaussians
    loss = -K.log(K.sum(alpha * pdf, axis=1, keepdims=True) + 1e-8)
    return loss

# Function to train MDN model for each country and save MAPE
def train_and_save_mape(X, y, target_name):
    country_mape = pd.DataFrame(index=countries.unique())

    for country in countries.unique():
        # Filter data for the current country
        country_mask = (countries == country)
        X_country = X[country_mask]
        y_country = y[country_mask]

        # Create and compile the MDN model
        mdn_model = create_mdn_model()

        # Implement early stopping
        early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

        # Train the MDN model on the current country's data
        mdn_model.fit(
            X_country, y_country,
            epochs=100,  # You can adjust the number of epochs
            batch_size=32,
            callbacks=[early_stopping],
            verbose=1
        )

        # Generate predictions on the MDN model
        y_pred = mdn_model.predict(X_country)

        # Calculate and save MAPE for the current country
        mape = mean_absolute_percentage_error(y_country, y_pred[:, :1].flatten())  # Use mean for prediction
        country_mape.loc[country, 'MAPE'] = mape

    # Save MAPE for each country as a CSV
    save_country_mape(country_mape, target_name)

# Train and save MAPE for TSS
train_and_save_mape(X_scaled, y_tss, 'Chla')




Epoch 1/100
5/5 [==============================] - 1s 3ms/step - loss: 16.9266
Epoch 2/100
5/5 [==============================] - 0s 3ms/step - loss: 16.0142
Epoch 3/100
5/5 [==============================] - 0s 3ms/step - loss: 15.2382
Epoch 4/100
5/5 [==============================] - 0s 3ms/step - loss: 14.3400
Epoch 5/100
5/5 [==============================] - 0s 3ms/step - loss: 13.5165
Epoch 6/100
5/5 [==============================] - 0s 3ms/step - loss: 12.6316
Epoch 7/100
5/5 [==============================] - 0s 3ms/step - loss: 11.6720
Epoch 8/100
5/5 [==============================] - 0s 3ms/step - loss: 10.6256
Epoch 9/100
5/5 [==============================] - 0s 3ms/step - loss: 9.6653
Epoch 10/100
5/5 [==============================] - 0s 3ms/step - loss: 8.7760
Epoch 11/100
5/5 [==============================] - 0s 3ms/step - loss: 7.9352
Epoch 12/100
5/5 [==============================] - 0s 3ms/step - loss: 7.3029
Epoch 13/100
5/5 [==============================] - 0

In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import numpy as np

# Function to save MAPE for each country
def save_country_mape(country_mape, target_name):
    country_mape.to_csv(f'{target_name}_country_mape.csv', index=True, header=['MAPE'])

# Define the MDN model creation function
def create_mdn_model(units=64, num_components=3, learning_rate=0.001):
    model = Sequential([
        Dense(units, activation='relu', input_dim=X_scaled.shape[1]),
        Dense(num_components * 3, activation='linear')  # 3 parameters per component
    ])

    def mdn_loss(y_true, y_pred):
        # Extract mean, log_sigma, and alpha using indexing
        num_params = num_components * 3
        mean = y_pred[:, :num_components]
        log_sigma = y_pred[:, num_components:2*num_components]
        alpha = y_pred[:, 2*num_components:]

        sigma = K.exp(log_sigma)

        # Gaussian probability density function
        pdf = K.exp(-0.5 * K.square((y_true - mean) / (sigma + 1e-8))) / (sigma + 1e-8) / np.sqrt(2 * np.pi)

        # Mixture of Gaussians
        loss = -K.log(K.sum(alpha * pdf, axis=1, keepdims=True) + 1e-8)
        return loss

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=mdn_loss)
    return model

# Load the data
data = pd.read_csv('CDOM_C.csv')

# Extract the 'Country' column
countries = data['Country']

# Select columns 2 to 10 for imputation
columns_for_imputation = data.columns[2:11]

# Impute NaN values for columns 2 to 10 using KNN
imputer = KNNImputer(n_neighbors=5)
data_imputed = data.copy()
data_imputed[columns_for_imputation] = imputer.fit_transform(data[columns_for_imputation])

# Check for and handle remaining NaN values
if data_imputed[columns_for_imputation].isnull().values.any():
    raise ValueError("NaN values still present after imputation.")

# Separate features and target variables
X = data_imputed.drop(columns=['Country', 'CDOM'])
y_cdom = data_imputed['CDOM']

# Standardize the features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Define the MDN loss function
num_components = 1
def mdn_loss(y_true, y_pred):
    # Extract mean, log_sigma, and alpha using indexing
    num_params = num_components * 3
    mean = y_pred[:, :num_components]
    log_sigma = y_pred[:, num_components:2*num_components]
    alpha = y_pred[:, 2*num_components:]

    sigma = K.exp(log_sigma)

    # Gaussian probability density function
    pdf = K.exp(-0.5 * K.square((y_true - mean) / (sigma + 1e-8))) / (sigma + 1e-8) / np.sqrt(2 * np.pi)

    # Mixture of Gaussians
    loss = -K.log(K.sum(alpha * pdf, axis=1, keepdims=True) + 1e-8)
    return loss

# Function to train MDN model for each country and save MAPE
def train_and_save_mape(X, y, target_name):
    country_mape = pd.DataFrame(index=countries.unique())

    for country in countries.unique():
        # Filter data for the current country
        country_mask = (countries == country)
        X_country = X[country_mask]
        y_country = y[country_mask]

        # Create and compile the MDN model
        mdn_model = create_mdn_model()

        # Implement early stopping
        early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

        # Train the MDN model on the current country's data
        mdn_model.fit(
            X_country, y_country,
            epochs=100,  # You can adjust the number of epochs
            batch_size=32,
            callbacks=[early_stopping],
            verbose=1
        )

        # Generate predictions on the MDN model
        y_pred = mdn_model.predict(X_country)

        # Calculate and save MAPE for the current country
        mape = mean_absolute_percentage_error(y_country, y_pred[:, :1].flatten())  # Use mean for prediction
        country_mape.loc[country, 'MAPE'] = mape

    # Save MAPE for each country as a CSV
    save_country_mape(country_mape, target_name)

# Train and save MAPE for CDOM
train_and_save_mape(X_scaled, y_cdom, 'CDOM')


Epoch 1/100
5/5 [==============================] - 2s 8ms/step - loss: nan
Epoch 2/100
5/5 [==============================] - 0s 4ms/step - loss: nan
Epoch 3/100
5/5 [==============================] - 0s 5ms/step - loss: nan
Epoch 4/100
5/5 [==============================] - 0s 5ms/step - loss: nan
Epoch 5/100
5/5 [==============================] - 0s 5ms/step - loss: nan
Epoch 6/100
5/5 [==============================] - 0s 5ms/step - loss: nan
Epoch 7/100
5/5 [==============================] - 0s 5ms/step - loss: nan
Epoch 8/100
5/5 [==============================] - 0s 4ms/step - loss: nan
Epoch 9/100
5/5 [==============================] - 0s 4ms/step - loss: nan
Epoch 10/100
5/5 [==============================] - 0s 2ms/step
Epoch 1/100
1/1 [==============================] - 1s 657ms/step - loss: 1.8045
Epoch 2/100
1/1 [==============================] - 0s 11ms/step - loss: 1.7230
Epoch 3/100
1/1 [==============================] - 0s 10ms/step - loss: 1.6468
Epoch 4/100
1/1 [======

4/4 [==============================] - 0s 4ms/step
Epoch 1/100
6/6 [==============================] - 1s 5ms/step - loss: nan
Epoch 2/100
6/6 [==============================] - 0s 3ms/step - loss: nan
Epoch 3/100
6/6 [==============================] - 0s 3ms/step - loss: nan
Epoch 4/100
6/6 [==============================] - 0s 3ms/step - loss: nan
Epoch 5/100
6/6 [==============================] - 0s 3ms/step - loss: nan
Epoch 6/100
6/6 [==============================] - 0s 3ms/step - loss: nan
Epoch 7/100
6/6 [==============================] - 0s 3ms/step - loss: nan
Epoch 8/100
6/6 [==============================] - 0s 3ms/step - loss: nan
Epoch 9/100
6/6 [==============================] - 0s 3ms/step - loss: nan
Epoch 10/100
6/6 [==============================] - 0s 3ms/step - loss: nan


6/6 [==============================] - 0s 3ms/step
Epoch 1/100
4/4 [==============================] - 1s 4ms/step - loss: nan
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: nan
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: nan
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 2.5895
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 2.3803
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 2.2128
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0771
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 1.9596
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 1.8562
Epoch 10/100
4/4 [==============================] - 0s 5ms/step - loss: 1.7589
Epoch 11/100
4/4 [==============================] - 0s 4ms/step - loss: 1.6705
Epoch 12/100
4/4 [==============================] - 0s 4ms/step - loss: 1.5846
Epoch 13/100
4/4 [=